In [90]:
import numpy as np
import cv2
from imutils import contours
from imutils import contours
import imutils
import myutils

In [91]:
# 指定输入图像和模板
input_img = cv2.imread('images/creditcard_identify/bestbut_3.jpg')
template = cv2.imread('images/creditcard_identify/digitjpg.jpg')

In [92]:
# 指定信用卡类型

FIRST_NUMBER = {
    '3':'American Express',
    '4':'Visa',
    '5':'MasterCard',
    '6':'Discover Card'
}

In [93]:
# 绘图显示
def cv_show(name,img):
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [94]:
# 显示模板
cv_show('template',template)
# 对模板进行灰度转换
ref = cv2.cvtColor(template,cv2.COLOR_BGR2GRAY)
cv_show('ref',ref)
# 对模板进行二值化
# 做轮廓检测时，输入的都是二值图像
# cv2.threshold()返回两个值，其中第二个参数是二值化结果
# cv2.threshold()参数：输入图像,阈值，超过阈值后赋予的值，二值化方法
# 超过10的取0，否则取255
ref = cv2.threshold(ref,15,255,cv2.THRESH_BINARY_INV)[1]
cv_show('ref',ref)

In [95]:
# 轮廓检测
refCnts,hiearchy = cv2.findContours(ref,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

In [96]:
# 画出轮廓
# 踩坑！ 这里一定不能在二值化后的图像上画轮廓，要copy原图然后画轮廓
ref_draw = template.copy()
cv2.drawContours(ref_draw,refCnts,-1,(0,0,255),2)
cv_show('ref_draw',ref_draw)
print(np.array(refCnts).shape)
# 对轮廓进行排序
refCnts = contours.sort_contours(refCnts,method='left-to-right')[0]
digits = {}
for (i,c) in enumerate(refCnts):
    (x,y,w,h) = cv2.boundingRect(c)
    roi = ref[y:y+h,x:x+w]
    roi = cv2.resize(roi,(57,88)) 
    # 计算外接矩形并且resize成合适大小
    digits[i] = roi
for i in range(len(digits)):
    cv_show('digits',digits[i])

(10,)


In [97]:
# 对输入图像进行处理
# 初始化卷积核
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(9,3))
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))

# 读取输入图像，预处理
cv_show('img',input_img)
input_img = imutils.resize(input_img,width=300)
cv_show('img',input_img)
gray = cv2.cvtColor(input_img,cv2.COLOR_BGR2GRAY)
cv_show('gray',gray)

In [98]:
# 礼帽操作，突出更明亮的区域
tophat = cv2.morphologyEx(gray,cv2.MORPH_TOPHAT,rectKernel)
cv_show('tophat',tophat)

# sobel算子
gradX = cv2.Sobel(tophat,ddepth=cv2.CV_32F,dx=1,dy=0,ksize=3)  # ksize=-1相当于用3*3的核
gradX = np.absolute(gradX)
# gradY = cv2.Sobel(tophat,ddepth=cv2.CV_32F,dx=0,dy=1,ksize=-1)
# gradY = np.absolute(gradY)
# grad = cv2.addWeighted(gradX,0.5,gradY,0.5,0)
(minVal,maxVal) = (np.min(gradX),np.max(gradX))
gradX = (255 * ((gradX-minVal) / (maxVal - gradX)))
gradX = gradX.astype("uint8")
cv_show('gradX',gradX)

c:\users\mrchan.000\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':


In [99]:
# 闭操作：先膨胀，再腐蚀
gradX = cv2.morphologyEx(gradX,cv2.MORPH_CLOSE,rectKernel)
cv_show('gradX',gradX)

# 二值化处理
thresh = cv2.threshold(gradX,0,255,
        cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
cv_show('THRESH',thresh)

thresh = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,sqKernel)
cv_show('thresh',thresh)

In [100]:
# 轮廓检测
threshCnt,_h = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

cnts = threshCnt

cur_img = input_img.copy()
cv2.drawContours(cur_img,cnts,-1,(0,0,255),2)
cv_show('cur_img',cur_img)

In [104]:
locs = []
for (i,c) in enumerate(cnts):
    (x,y,w,h) = cv2.boundingRect(c)
    ar = w / float(h)
    # 选择合适的区域
    if ar > 2.5 and ar < 4.0:
        if(w > 40 and w < 55) and (h > 10 and h < 20):
            # 符合的留下来
            locs.append((x,y,w,h))
locs = sorted(locs,key=lambda x:x[0])
output = []  

In [ ]:

for (i,(gX,gY,gW,gH)) in enumerate(locs):
    groupOutput = []   
    group = gray[gY -5:gY +gH +5,gX - 5:gX +gW + 5]
    cv_show('group',group)
    group = cv2.threshold(group,0,255,
                          cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    cv_show('group',group)
    
    # 计算每一组的轮廓
    digitCnts,_h = cv2.findContours(group.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    digitCnts = contours.sort_contours(digitCnts,method='left-to-right')[0]
    for c in digitCnts:
        (x,y,w,h) = cv2.boundingRect(c)
        print(x,y,w,h)   
        roi = group[y:y+h,x:x+w]
        roi = cv2.resize(roi,(57,88))
        cv_show('roi',roi)
        
        scores = []
        
        for (digit,digitRIO) in digits.items():
            result = cv2.matchTemplate(roi,digitRIO,cv2.TM_CCOEFF)
            (_,score,_,_) = cv2.minMaxLoc(result)
            scores.append(score) 
        
        groupOutput.append(str(np.argmax(scores)))
        
        cv2.rectangle(input_img,(gX-5,gY-5),(gX +gW +5,gY +gH +5),(0,0,255),1)
        cv2.putText(input_img,''.join(groupOutput),(gX,gY -15),cv2.FONT_HERSHEY_SIMPLEX,0.65,(0,0,255),2)
        
        output.extend(groupOutput)

In [103]:
cv2.imshow('Image',input_img)
